In [ ]:
import duckdb
from scraper import get_trends_all, get_jobs_all
import pandas as pd

# Define the jobs to scrape
jobs_to_scrape = [
    "data analyst",
    # "data scientist",
    # "data engineer"
]

# Define the locations
locations  = [
    "Sydney",
    # "Melbourne",
    # "Brisbane",
    # "Perth",
    # "Adelaide",
    # "Canberra",
    # "Australia"
]

# get the trends
trends = get_jobs_all(jobs_to_scrape, locations)

# read the trends into a dataframe
table_name = "jobs"

with duckdb.connect('jobs.db') as con:
    con.register('new_jobs', trends)
    con.execute(f"CREATE or REPLACE TABLE {table_name} AS SELECT * FROM new_jobs")
    df = con.execute(f"SELECT * FROM {table_name}").fetchdf()
    df.head()

In [ ]:
df.head()

In [ ]:
from duckdb import CatalogException
# check if table exists in duck db
try:
    table_name = "jobs"
    with duckdb.connect('jobs.db') as con:
        df = con.execute(f"SELECT * FROM {table_name}").fetchdf()
        print(df.head())
except CatalogException as e:
    if 'does not exist' in str(e):
        print(f"Table {table_name} does not exist. Need to create it first.")
    else:
        raise e


In [ ]:
table_name = "jobs_descriptions"

with duckdb.connect('jobs.db') as con:
    df = con.execute(f"SELECT * FROM {table_name}").fetchdf()
    df.head()

In [ ]:
df

In [ ]:
new_table = pd.DataFrame(
    {
        "job_title": ["data analyst", "data scientist", "data engineer"],
        "description": ["data analyst description", "data scientist description", "data engineer description"]
    }
)

with duckdb.connect('jobs.db', read_only=False) as con:
    con.register('new_table', new_table)
    con.execute(f"CREATE or REPLACE TABLE {table_name} AS SELECT * FROM new_table")
    df = con.execute(f"SELECT * FROM {table_name}").fetchdf()

In [ ]:
import requests
request_base_url = r"https://www.yelp.com.au/gql/batch"
payload = [{"operationName":"GetBusinessReviewFeed","variables":{"encBizId":"30UX2wFjo_jK82LQVbW1-g",
                                                                 "reviewsPerPage":30,
                                                                 "selectedReviewEncId":"",
                                                                 "hasSelectedReview":False,
                                                                 "sortBy":"RELEVANCE_DESC",
                                                                 "languageCode":"en","ratings":[5],
                                                                 "queryText":"",
                                                                 "isSearching":False,
                                                                 "after":"eyJ2ZXJzaW9uIjoxLCJ0eXBlIjoib2Zmc2V0Iiwib2Zmc2V0Ijo5fQ==",
                                                                 "isTranslating":False,
                                                                 "translateLanguageCode":"en",
                                                                 "reactionsSourceFlow":"businessPageReviewSection",
                                                                 "guv":"2337E94512339811",
                                                                 "minConfidenceLevel":"HIGH_CONFIDENCE",
                                                                 "highlightType":"",
                                                                 "highlightIdentifier":"",
                                                                 "isHighlighting":False},
                                                                 "extensions":{"operationType":"query",
                                                                               "documentId":"ef51f33d1b0eccc958dddbf6cde15739c48b34637a00ebe316441031d4bf7681"}}]
response = requests.post(request_base_url,
                        json=payload,
                        headers={'User-Agent': 'Mozilla/5.0'}
                        )
print(response.json())


data = response.json()[0]['data']
count = 0
for edge in data['business']['reviews']['edges']:
    print(edge['node']['text']['full'])
    print(edge['node']['rating'])
    count +=1

print(count)

In [7]:
import requests
from bs4 import BeautifulSoup

url = "https://www.seek.com.au/Data-Analyst-jobs/in-All-Sydney-NSW"

response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

# find all the job cards
job_cards = soup.find_all('article')

for job in job_cards:
    title = job.find('a', {'data-automation': 'jobTitle'}).text
    company = job.find('a', {'data-automation': 'jobCompany'}).text
    print(f"Title: {title}")
    print(f"Company: {company}")


Title: Data Engineer / Analyst
Company: Just Better Care Australia Pty Ltd
Title: Data Analyst
Company: The Data School Pty Ltd
Title: Data Analyst (APS6)
Company: Department of Education
Title: Data Analyst Metering
Company: Sydney Water
Title: Data Analyst
Company: Crown Equipment Pty Limited
Title: Data Engineer / Analyst
Company: Just Better Care Australia Pty Ltd
Title: Data Analyst, Financial Crime
Company: Cuscal Limited (SR)
Title: BI Data Analyst
Company: Talent Insights Group Pty Ltd
Title: Customer Insights/Data Analyst
Company: Ausgrid
Title: Senior Commercial Data Analyst
Company: InvoCare Australia Pty Limited
Title: Data Analyst
Company: Prosperity Media
Title: HR Data Analyst & Administrator
Company: Crown Equipment Pty Limited
Title: Senior Data Analyst
Company: Preacta Recruitment
Title: Business Analyst | Data Governance | Insurance
Company: Robert Half
Title: Data Analyst and Data Acquisition
Company: Department of Health & Aged Care
Title: Senior Data Analyst
Compa

In [11]:
url = "https://www.seek.com.au/api/chalice-search/v4/search?&where=All+Melbourne+VIC&page=2&keywords=Data+Analyst"

response = requests.get(url)

data = response.json()

import pandas as pd

df = pd.DataFrame(data['data'])

df.head()

,advertiser,area,areaId,areaWhereValue,automaticInclusion,bulletPoints,classification,companyName,companyProfileStructuredDataId,displayStyle,...,suburbId,suburbWhereValue,teaser,title,tracking,workType,workArrangements,isPrivateAdvertiser,branding,tags
0,"{'id': '30675127', 'description': 'Aged Care Q...",CBD & Inner Suburbs,5069,Melbourne CBD & Inner Suburbs Melbourne VIC,False,"[Flexible Working Arrangements, In addition, 1...","{'id': '6281', 'description': 'Information & C...",Aged Care Quality and Safety Commission,549593,{'search': 'A'},...,18578,Melbourne VIC 3000,Data/Senior Data Analysts – Multiple opportuni...,Data/Senior Data Analysts,ewogICJ0b2tlbiI6ICJmYjMzYzY3Yi00MTVhLTQ1ZTktYW...,Full time,"{'data': [{'id': '1', 'label': {'text': 'On-si...",False,NaN,NaN
1,"{'id': '61910210', 'description': 'Peroptyx'}",CBD & Inner Suburbs,5069,Melbourne CBD & Inner Suburbs Melbourne VIC,False,"[Work from Home, Flexible Hours, Incentives]","{'id': '6281', 'description': 'Information & C...",Peroptyx,714726,{'search': 'A'},...,18578,Melbourne VIC 3000,"At Peroptyx, we are looking for Data Analysts ...",Data Analyst - WFH - No Experience Required,ewogICJ0b2tlbiI6ICJmYjMzYzY3Yi00MTVhLTQ1ZTktYW...,Part time,"{'data': [{'id': '3', 'label': {'text': 'Remot...",False,{'id': 'f4f8e1aa-1b23-46ba-bb08-9d264730f199.1...,"[{'type': 'EXPIRES_SOON', 'label': 'Expiring s..."
2,"{'id': '46760079', 'description': 'BlueBet'}",CBD & Inner Suburbs,5069,Melbourne CBD & Inner Suburbs Melbourne VIC,False,[],"{'id': '1223', 'description': 'Science & Techn...",NaN,0,NaN,...,18578,Melbourne VIC 3000,This is your chance to shape the future of the...,Data Scientist,ewogICJ0b2tlbiI6ICJmYjMzYzY3Yi00MTVhLTQ1ZTktYW...,Full time,"{'data': [{'id': '1', 'label': {'text': 'On-si...",False,NaN,NaN
3,"{'id': '1773943', 'description': 'Paccar Austr...",Eastern Suburbs,5071,Eastern Suburbs Melbourne VIC,False,"[Friendly team environment, Excellent salary p...","{'id': '1200', 'description': 'Accounting'}",Paccar Inc,18447,{'search': 'A'},...,17214,Bayswater VIC 3153,Exciting opportunity to join our Finance Team!...,Accounts Reconciliation & Data Analyst,ewogICJ0b2tlbiI6ICJmYjMzYzY3Yi00MTVhLTQ1ZTktYW...,Full time,"{'data': [{'id': '1', 'label': {'text': 'On-si...",False,{'id': 'a149f00f-d16f-a79e-7c3e-f901b717bcff.1...,NaN
4,"{'id': '', 'description': 'Private Advertiser'}",CBD & Inner Suburbs,5069,Melbourne CBD & Inner Suburbs Melbourne VIC,False,"[Generous benefits and discounts included., Ca...","{'id': '1200', 'description': 'Accounting'}",Private Advertiser,0,{'search': 'A'},...,18578,Melbourne VIC 3000,Play a role in developing automated reporting ...,Retail Analyst,ewogICJ0b2tlbiI6ICJmYjMzYzY3Yi00MTVhLTQ1ZTktYW...,Full time,"{'data': [{'id': '1', 'label': {'text': 'On-si...",True,NaN,NaN


: 